In [1]:
import numpy as np
import g3_utils as ut
import matplotlib.pyplot as plt
from spt3g import core
import pathlib

In [2]:
control_computer_g3_dir = pathlib.Path("/media/player1/blast2020fc1/blasttng_g3")
roach1_pass3_file = control_computer_g3_dir / "testing/roach1_pass3.g3"

ra_df_added = control_computer_g3_dir / "mapmaking/ra_df_added.g3"
norm_df_added = control_computer_g3_dir / "mapmaking/norm_df_added.g3"

In [3]:
kid_rejects = [11, 21, 57, 81, 111, 127, 147, 148, 149, 150, 151, 152, 154, 158, 170, 181, 182, 192, 195,
              211, 223, 225, 245, 263, 282, 286, 293, 297, 319, 327, 331, 333, 334, 336, 337, 340, 341, 349, 352]
exclude_kids = [ut.kid_string(reject_id, roach_id=1) for reject_id in kid_rejects]

In [4]:
# create non-normalized df and compute stats
stats = ut.DetectorStats(data_key="df")
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(roach1_pass3_file))
pipe.Add(ut.AddScanDF, exclude_kids=exclude_kids)
pipe.Add(stats)
pipe.Add(ut.FrameCounter)
pipe.Run()
# see signal_analysis.ipynb for plots of this data
detector_medians = np.median(np.array(stats.medians), axis=0)
detector_stds = np.median(np.array(stats.stds), axis=0)


Calibration
PipelineInfo
Scan (x117)
EndProcessing


In [5]:
# Uncomment this pipeline to recompute & overwrite ra_df_added/norm_df_added

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(roach1_pass3_file))
pipe.Add(ut.add_radec_so3g)
pipe.Add(ut.AddScanDF, exclude_kids=exclude_kids)
pipe.Add(ut.add_cal_lamp_df, iq_key="cal_lamp_data", exclude_kids=exclude_kids)
pipe.Add(core.G3Writer, filename=str(ra_df_added))
pipe.Add(ut.NormalizeDF, detector_medians=detector_medians)
pipe.Add(core.G3Writer, filename=str(norm_df_added))
pipe.Add(ut.FrameCounter)
pipe.Run()

"""
Pipeline profiling results:
spt3g.core.G3Reader: 0.071389 user, 0.037763 system, 120 frames (0.000910 s per input frame)
_pipelineinfo: 0.000000 user, 0.000000 system, 120 frames (0.000000 s per input frame)
g3utils.add_radec_so3g: 0.197547 user, 0.000434 system, 120 frames (0.001650 s per input frame)
g3utils.AddScanDF: 25.434544 user, 0.064807 system, 120 frames (0.212495 s per input frame)
g3utils.add_cal_lamp_df: 0.281990 user, 0.009692 system, 120 frames (0.002431 s per input frame)
g3utils.NormalizeDF: 0.253910 user, 0.000002 system, 120 frames (0.002116 s per input frame)
g3utils.FrameCounter: 0.003378 user, 0.000064 system, 120 frames (0.000029 s per input frame)
spt3g.core.G3Writer: 3.151443 user, 0.251032 system, 120 frames (0.028354 s per input frame)
Total: 29.394201 user, 0.363794 system
Peak memory consumption (369.4 MB) in module g3utils.add_cal_lamp_df
""";


Calibration
PipelineInfo

WARN (Unknown) 16-Apr-2025:15:18:46 PDT: Exception in module "g3_utils.coords.add_radec_so3g" (G3Pipeline.cxx:124 in size_t {anonymous}::PushFrameThroughQueue(G3FramePtr, bool, bool, rusage&, std::vector<{anonymous}::G3Pipeline_mod_data>&, std::vector<{anonymous}::G3Pipeline_mod_data>::iterator, int&, std::deque<{anonymous}::G3Pipeline_proc_data>&, G3FramePtr&))


ArgumentError: Python argument types in
    None.None(G3Timestream, numpy.float64)
did not match C++ signature:
    None(G3Timestream {lvalue}, G3Time)

In [ ]:
def first_frame(file, type=core.G3FrameType.Scan):
    grabber = ut.FirstFrameGrabber(frame_type=type)
    pipe = core.G3Pipeline()
    pipe.Add(core.G3Reader, filename=file)
    pipe.Add(grabber)
    pipe.Run()
    return grabber.first_frame

kids: np.ndarray[str] = np.array(first_frame(str(norm_df_added))["norm_df"].names)
kids.sort()

In [ ]:
# center of the sky map
ra0 = 231.15 * core.G3Units.deg
dec0 = -56.2 * core.G3Units.deg

# map dimensions
xlen = 1.4 * core.G3Units.deg
ylen = 0.9 * core.G3Units.deg

# pixel resolution
res = 1 * core.G3Units.arcmin

In [ ]:
source_coords = {}
# determine source shifts
binners = [
    ut.SingleMapBinner(
        kid,
        timestreams="df",
        ra0=ra0,
        dec0=dec0,
        xlen=xlen,
        ylen=ylen,
        res=res
    ) for kid in kids
]

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(norm_df_added))
for binner in binners:
    pipe.Add(binner)
pipe.Add(ut.FrameCounter)
pipe.Run()

for kid, binner in zip(kids, binners):
    source_coords[kid] = binner.source_coords()

In [ ]:
binner = ut.MapBinner(
    timestreams="df_ctremoved",
    source_coords=source_coords,
    stds=detector_stds,
    ra0=ra0, dec0=dec0, xlen=xlen, ylen=ylen, res=res,
)

# create the pipeline
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(norm_df_added))
pipe.Add(ut.remove_common_mode, in_key="norm_df")
pipe.Add(binner)
pipe.Add(ut.FrameCounter)
pipe.Run()

In [ ]:
plt.imshow(binner.data, origin='lower')
plt.xticks(range(binner.nx + 1)[::80], [f"{ra:.2f}" for ra in binner.ra_edges[::80] / core.G3Units.deg],
               rotation=45)
plt.yticks(range(binner.ny + 1)[::80], [f"{dec:.2f}" for dec in binner.dec_edges[::80] / core.G3Units.deg])
plt.colorbar()
plt.xlabel("RA (deg)")
plt.ylabel("DEC (deg)")
plt.show()

plt.imshow(binner.hits, origin='lower')
plt.xticks(range(binner.nx + 1)[::80], [f"{ra:.2f}" for ra in binner.ra_edges[::80] / core.G3Units.deg],
               rotation=45)
plt.yticks(range(binner.ny + 1)[::80], [f"{dec:.2f}" for dec in binner.dec_edges[::80] / core.G3Units.deg])
plt.colorbar()
plt.title("# of Hits")
plt.xlabel("RA (deg)")
plt.ylabel("DEC (deg)")
plt.show()


In [ ]:
import matplotlib.colors as colors
with np.errstate(invalid='ignore'):
    m = binner.data / binner.hits
linthresh = 0.02
plt.imshow(m, origin='lower', norm=colors.SymLogNorm(linthresh))
plt.xticks(range(binner.nx + 1)[::10], [f"{ra:.2f}" for ra in binner.ra_edges[::10] / core.G3Units.deg],
               rotation=45)
plt.yticks(range(binner.ny + 1)[::10], [f"{dec:.2f}" for dec in binner.dec_edges[::10] / core.G3Units.deg])
plt.colorbar()
plt.xlabel("RA (deg)")
plt.ylabel("DEC (deg)")
plt.title(f"Roach 1 Pass 3, Combined Map\nLog scale where |DF| > {linthresh}")
plt.show()

In [ ]:
def simulate_astronomical(frame, prev_map=None, ra0=None, dec0=None, xlen=None, ylen=None, res=None):
    if frame.type != core.G3FrameType.Scan:
        return
    
    X = np.array(frame["ra"])
    Y = np.array(frame["dec"])

    nx = int(xlen / res)
    ny = int(ylen / res)
    ra_edges = np.linspace(-xlen / 2, xlen / 2, nx + 1) + ra0
    dec_edges = np.linspace(-ylen / 2, ylen / 2, ny + 1) + dec0
    
    # Step 1: Digitize X and Y to find bin indices
    x_bin_indices = np.digitize(X, ra_edges) - 1  # Subtract 1 because np.digitize gives 1-based index
    y_bin_indices = np.digitize(Y, dec_edges) - 1  # Subtract 1 for same reason

    # Step 2: Initialize an array to store the reconstructed weights
    reconstructed_W = np.zeros_like(X)

    # Step 3: Loop over each weight, and assign it to the corresponding bin
    for i in range(len(X)):
        # Get the corresponding bin in the 2D histogram M
        x_bin = x_bin_indices[i]
        y_bin = y_bin_indices[i]

        # Reconstruct the weight by dividing the sum in the bin by the number of points in the bin
        # (to reverse the histogram summing)
        reconstructed_W[i] = prev_map[y_bin, x_bin] / np.sum((x_bin_indices == x_bin) & (y_bin_indices == y_bin))

    sim_ast = core.G3Timestream(reconstructed_W)
    sim_ast.start = frame["ra"].start
    sim_ast.stop = frame["ra"].stop
    frame["sim_ast"] = sim_ast


pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(norm_df_added))
pipe.Add(simulate_astronomical, prev_map=m, ra0=ra0, dec0=dec0, xlen=xlen, ylen=ylen, res=res)
pipe.Add(ut.FrameCounter)
pipe.Run()